<a href="https://colab.research.google.com/github/Gustavo-br-rj/PROJ-BI_MASTER/blob/main/3_SALVA_MODELOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Previsão de Série Temporal da Temperatura de Mancais Utilizando LSTM

## Parte 0 - Importação das Bibliotecas


In [ ]:
#%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import math

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

#tf.__version__

## Parte 1 - Importação do Dataset

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/BI_MASTER/PROJETO_FINAL_BIMASTER/DATASETS/dataset_C-4451_08001A.csv')

#Conversão do campo data para datetime e ordenação do dataset
dataset['DATA'] = pd.to_datetime(dataset['DATA'],dayfirst=True)
dataset = dataset.sort_values(['EQUIPAMENTO', 'MANCAL', 'DATA'], ascending=[1, 1, 1])

## Parte 2 - Declaração de Funções

In [ ]:
# Função para retornar os melhores parâmetros para a construção da RN do mancal
def returnBestParams(file, metric):
  #RMSE, MSE, MAPE
  if ( metric == 'RMSE' or metric == 'MSE' or metric == 'MAPE' ):
    bestParams = file.loc[file[metric] == file[metric].min()]

  return bestParams

In [ ]:
# Função para criar a estrutura de dados com janela WINDOW e output OUTPUTSIZE para entrada na RN
def prepareSetRN(windowSize, outputSize, flatSet ):
  preparedX = []
  preparedY = []
  setSize = len(flatSet)
  for i in range(windowSize, setSize - (outputSize-1)):
    preparedX.append(flatSet[i-windowSize:i, 0])
    preparedY.append(flatSet[i:i + outputSize, 0])

  preparedX, preparedY = np.array(preparedX), np.array(preparedY)
  return preparedX, preparedY

In [ ]:
# Função para criar a RN com base nos parâmetros informados
def criaRN(outputParam, layerParam, unitParam, dropoutParam, optmizerParam, X_trainParam):
  # Inicializar a RNN
  regressor = Sequential()

  # Adicionar a primeira camada LSTM e Dropout 
  regressor.add(LSTM(units = unitParam[0], return_sequences = (layerParam > 1), input_shape = (X_trainParam.shape[1], 1)))
  regressor.add(Dropout(dropoutParam))
 
  if ( layerParam >= 2 ):
    # Adicionar a segunda camada LSTM e Dropout
    regressor.add(LSTM(units = unitParam[1], return_sequences = (layerParam > 2)))
    regressor.add(Dropout(dropoutParam))

  if ( layerParam >= 3):
    # Adicionar a terceira camada LSTM e Dropout
    regressor.add(LSTM(units = unitParam[2], return_sequences = (layerParam > 3)))
    regressor.add(Dropout(dropoutParam))

  # camada de saída
  regressor.add(Dense(units = outputParam, activation='linear'))
  #Testar sigmoid e linear

  # Compilar a rede
  regressor.compile(optimizer = optmizerParam, loss = 'mean_squared_error', metrics=['accuracy'])

  # Visualizar a rede
  #regressor.summary()

  return regressor

# 3 - Obtendo os melhores parâmetros encontrados para a criação da RN de cada Mancal 

In [ ]:
# Importação dos resultados da avaliação dos parâmetros da RN para obtenção do melhor resultado para cada MANCAL
path = '/content/drive/MyDrive/BI_MASTER/PROJETO_FINAL_BIMASTER/AVALIACAO_REDES/'

files = os.listdir(path)

for f in files:
  if f.endswith("_results.csv"):
    result4RN = pd.read_csv(path + f)
    bestParams = returnBestParams(result4RN, 'RMSE' )

    equipamento = bestParams.iloc[0]['EQUIPAMENTO']
    mancal      = bestParams.iloc[0]['MANCAL']
    window      = bestParams.iloc[0]['WINDOW']
    output      = bestParams.iloc[0]['OUTPUT']
    layer       = bestParams.iloc[0]['NUM. CAMADAS']
    unit1       = bestParams.iloc[0]['NUM. NEURONIOS CAMADA 1']
    unit2       = bestParams.iloc[0]['NUM. NEURONIOS CAMADA 2']
    unit3       = bestParams.iloc[0]['NUM. NEURONIOS CAMADA 3']
    unit        = [unit1,unit2,unit3]
    dropout     = bestParams.iloc[0]['DROPOUT']
    optmizer    = bestParams.iloc[0]['OTIMIZADOR']
    epochs      = bestParams.iloc[0]['EPOCHS']
    batch_size  = bestParams.iloc[0]['BATCH_SIZE']

    datasetMancal = dataset.loc[(dataset['MANCAL'] == mancal)]
    #Calculando tamanho dos conjuntos de treino e teste
    train_size = math.ceil(len(datasetMancal)*0.85)
    test_size  = len(datasetMancal) - train_size

    #Separando conjuto de treino e teste apenas com valor da série
    train_set = datasetMancal.iloc[0:train_size, 3:4].values
    test_set  = datasetMancal.iloc[train_size:(train_size + test_size), 3:4].values

    #Normalização
    sc = MinMaxScaler(feature_range = (0, 1))
    training_set_scaled = sc.fit_transform(train_set)

    X_train = []
    y_train = []
    
    X_train, y_train = prepareSetRN(window, output, training_set_scaled)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

    regressor = criaRN(output, layer, unit, dropout, optmizer, X_train)
    regressor.fit(X_train, y_train, epochs = epochs, batch_size = batch_size)

    #Salvar modelo
    dirModels = '/content/drive/MyDrive/BI_MASTER/PROJETO_FINAL_BIMASTER/MODELOS/'
    nomeModelo = '{0}{1}__{2}_model'.format(dirModels,equipamento, mancal)
    regressor.save(nomeModelo)

Epoch 1/800
4/4 [==============================] - 5s 13ms/step - loss: 0.2797 - accuracy: 0.3056
Epoch 2/800
4/4 [==============================] - 0s 12ms/step - loss: 0.2551 - accuracy: 0.3426
Epoch 3/800
4/4 [==============================] - 0s 12ms/step - loss: 0.2249 - accuracy: 0.3333
Epoch 4/800
4/4 [==============================] - 0s 12ms/step - loss: 0.1837 - accuracy: 0.3333
Epoch 5/800
4/4 [==============================] - 0s 12ms/step - loss: 0.1319 - accuracy: 0.3333
Epoch 6/800
4/4 [==============================] - 0s 12ms/step - loss: 0.0710 - accuracy: 0.3333
Epoch 7/800
4/4 [==============================] - 0s 11ms/step - loss: 0.0567 - accuracy: 0.3056
Epoch 8/800
4/4 [==============================] - 0s 11ms/step - loss: 0.0568 - accuracy: 0.3333
Epoch 9/800
4/4 [==============================] - 0s 12ms/step - loss: 0.0483 - accuracy: 0.3519
Epoch 10/800
4/4 [==============================] - 0s 11ms/step - loss: 0.0540 - accuracy: 0.3333
Epoch 11/800
4/4 [=

INFO:tensorflow:Assets written to: /content/drive/MyDrive/BI_MASTER/PROJETO_FINAL_BIMASTER/MODELOS/C-4451.08001A__EJA1.A-TC23.F_CV_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/BI_MASTER/PROJETO_FINAL_BIMASTER/MODELOS/C-4451.08001A__EJA1.A-TC23.F_CV_model/assets


Epoch 1/800
4/4 [==============================] - 2s 8ms/step - loss: 0.6009 - accuracy: 0.5094
Epoch 2/800
4/4 [==============================] - 0s 7ms/step - loss: 0.4517 - accuracy: 0.5189
Epoch 3/800
4/4 [==============================] - 0s 7ms/step - loss: 0.3342 - accuracy: 0.5189
Epoch 4/800
4/4 [==============================] - 0s 7ms/step - loss: 0.2269 - accuracy: 0.5189
Epoch 5/800
4/4 [==============================] - 0s 7ms/step - loss: 0.1363 - accuracy: 0.5094
Epoch 6/800
4/4 [==============================] - 0s 7ms/step - loss: 0.0654 - accuracy: 0.5000
Epoch 7/800
4/4 [==============================] - 0s 7ms/step - loss: 0.0393 - accuracy: 0.5000
Epoch 8/800
4/4 [==============================] - 0s 7ms/step - loss: 0.0359 - accuracy: 0.5283
Epoch 9/800
4/4 [==============================] - 0s 7ms/step - loss: 0.0357 - accuracy: 0.4245
Epoch 10/800
4/4 [==============================] - 0s 8ms/step - loss: 0.0297 - accuracy: 0.5755
Epoch 11/800
4/4 [===========

INFO:tensorflow:Assets written to: /content/drive/MyDrive/BI_MASTER/PROJETO_FINAL_BIMASTER/MODELOS/C-4451.08001A__EJA1.A-TC21.F_CV_model/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/BI_MASTER/PROJETO_FINAL_BIMASTER/MODELOS/C-4451.08001A__EJA1.A-TE1209B.F_CV_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/BI_MASTER/PROJETO_FINAL_BIMASTER/MODELOS/C-4451.08001A__EJA1.A-TE1209B.F_CV_model/assets


Epoch 1/800
4/4 [==============================] - 5s 32ms/step - loss: 0.4755 - accuracy: 0.3143
Epoch 2/800
4/4 [==============================] - 0s 21ms/step - loss: 0.3839 - accuracy: 0.2857
Epoch 3/800
4/4 [==============================] - 0s 31ms/step - loss: 0.2534 - accuracy: 0.3333
Epoch 4/800
4/4 [==============================] - 0s 30ms/step - loss: 0.1196 - accuracy: 0.3238
Epoch 5/800
4/4 [==============================] - 0s 20ms/step - loss: 0.0850 - accuracy: 0.3429
Epoch 6/800
4/4 [==============================] - 0s 26ms/step - loss: 0.0724 - accuracy: 0.3333
Epoch 7/800
4/4 [==============================] - 0s 26ms/step - loss: 0.0711 - accuracy: 0.4000
Epoch 8/800
4/4 [==============================] - 0s 21ms/step - loss: 0.0627 - accuracy: 0.2952
Epoch 9/800
4/4 [==============================] - 0s 22ms/step - loss: 0.0519 - accuracy: 0.2952
Epoch 10/800
4/4 [==============================] - 0s 29ms/step - loss: 0.0537 - accuracy: 0.3048
Epoch 11/800
4/4 [=

INFO:tensorflow:Assets written to: /content/drive/MyDrive/BI_MASTER/PROJETO_FINAL_BIMASTER/MODELOS/C-4451.08001A__EJA1.A-TE1206B.F_CV_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/BI_MASTER/PROJETO_FINAL_BIMASTER/MODELOS/C-4451.08001A__EJA1.A-TE1206B.F_CV_model/assets
